# ACM + Lyft HackNight - Python Starter Notebook

<strong>Prompt:</strong> In a city of only taxis from the Bauer Taxi Service, your team is put in charge to earn Lyft more business and win riders over. The ACM Executive team has provided you with this starter notebook, which showcases how you can utilize the RESTful API that has 5 public endpoints that you can make calls to. These endpoints are listed below:

<ul>
    <li><strong>/time:</strong> GET</li>
        <ul>
            <li>Returns the simulation period you are developing your pricing model for.</li>
        </ul>
    <li><strong>/trips:</strong> GET</li>
        <ul>
            <li>Returns the taxi trips that occured with the criterion specified.</li>
        </ul>
    <li><strong>/count:</strong> GET</li>
        <ul>
            <li>Returns the total amount of trips that occured during a certain time period.</li>
        </ul>
    <li><strong>/pricing:</strong> POST</li>
        <ul>
            <li>Submits your team's general pricing strategy.</li>
        </ul>
    <li><strong>/zones:</strong> POST</li>
        <ul>
            <li>Submits the second component of your pricing strategy: the areas you believe are power zones.</li>
        </ul>
</ul>

<strong>Game Overview:</strong> There are 5 simple steps you need to follow to succeed this HackNight!
<ol>
    <li>Wait for a "simulation" (access to taxi data of a specific week and prior)</li>
    <li>Use the endpoints with associated GET requests to do data analysis to develop your pricing model.</li>
    <li>Submit your pricing model</li>
    <li>View your revenue generated (the results) on the diagram shown at the end of each simulation.</li>
    <li>Repeat this process for every simulation</li>
</ol>

## Learning Objective: Using a RESTful API

An API (Application Programming Interface) is a set of methods that controls the access a programmer has to what they want to reach (in this case, the taxi dataset). Here, the Lyft HackNight API limits access to taxi data by bounding it to the simulation. What does this mean? It means that you can only access taxi data of a certain date or date range when it is within the simulation date range or prior.

Now, let's get started with set up process.

### Initial Setup and Imports

In [17]:
# Required imports
import requests # Python's go-to HTTP request library
from datetime import datetime

In [18]:
# Environment and other variables for the API to know who you are
URL = "https://lyft-vingkan.c9users.io"
TEAM = "lyft09"

### Our first GET request: /time

Format of a GET request with Python's request library:

requests.get(SERVER_URL + "/ENDPOINT_NAME/", params="{QUERY HERE}).json()

In [19]:
requests.get(URL + "/time/").json()

{'message': 'The simulation is not over, the time is 11/3/2019.',
 'sucess': True,
 'time': 1572739200000}

Note that the /time endpoint doesn't take any parameters, so there's no need to specify "params" in the method call. However, in the /trips endpoint below, we'll do just that.

### /trips endpoint

The /trips endpoint allows us to find all trips that have occured in a specific time range. Note, however, that you can only search for trips that happen in the current simulation. We don't want you to predict the future!

<strong>Endpoint: /trips</strong>
<ul>
    <li>team (str): the string name of your team as indicated in the "TEAM" variable above</li>
    <li>start (str): the date and time you want to start searching from, in format %m/%d/%Y %H:%M A/PM</li>
        <ul>
            <li>Example: "10/07/2017 5:00PM"</li>
        </ul>
    <li>end (str): the date and time you want to end the search on, in format %m/%d/%Y %H:%M A/PM</li>
    <li>limit (int): maximum number of results to return</li>
    <li>offset (int): indicates you only want to receive data points past this index</li>
</ul>

In [24]:
# Build the query
trips_query = {
    "team": TEAM,
    "start": "10/27/2019 12:00 AM",
    "end": "10/29/2019 11:59 PM",
    "limit": 1
}

# Send the request
trips_response = requests.get(URL + "/trips/", params=trips_query).json()
trips_response

{'length': 1,
 'response': [{'company': 'Bauer Taxi Service',
   'dropoff_centroid_latitude': '41.898331794',
   'dropoff_centroid_longitude': '-87.620762865',
   'dropoff_community_area': '8',
   'entry_idx': 0,
   'extras': '0',
   'fare': '6.5',
   'pickup_centroid_latitude': '41.884987192',
   'pickup_centroid_longitude': '-87.620992913',
   'pickup_community_area': '32',
   'taxi_id': '1f601da85fba049c4ca4363c0bad4e6a96a5f72fe0beace329d0db5542b724892aa9a869def673292002803067a24c31bee67f9011dfe7a3c57ac0fe9a709722',
   'tips': '1.5',
   'tolls': '0',
   'trip_end_timestamp': '2019-10-28T17:30:00',
   'trip_id': '00007bf6442eddf62c3e4dbe198fdc4c2e09ccbb',
   'trip_miles': '1',
   'trip_seconds': '480',
   'trip_start_timestamp': '2019-10-28T17:15:00',
   'trip_total': '8.5'}],
 'success': True}

The JSON (Javascript Object Notation) object we got back from our call to /trips is 1 ride that occured from 2:00 - 3:00PM. In Python, this JSON object is interpreted as a dictionary. <strong>We care about the "response" key.</strong> To retrieve the response, we simply use Python's dictionary bracket notation, which will give us the datapoint we want.

In [25]:
# Gives us a list of the taxi rides that fits the criterion we wanted. Only one in here since we set limit = 1.
trips_response["response"]

[{'company': 'Bauer Taxi Service',
  'dropoff_centroid_latitude': '41.898331794',
  'dropoff_centroid_longitude': '-87.620762865',
  'dropoff_community_area': '8',
  'entry_idx': 0,
  'extras': '0',
  'fare': '6.5',
  'pickup_centroid_latitude': '41.884987192',
  'pickup_centroid_longitude': '-87.620992913',
  'pickup_community_area': '32',
  'taxi_id': '1f601da85fba049c4ca4363c0bad4e6a96a5f72fe0beace329d0db5542b724892aa9a869def673292002803067a24c31bee67f9011dfe7a3c57ac0fe9a709722',
  'tips': '1.5',
  'tolls': '0',
  'trip_end_timestamp': '2019-10-28T17:30:00',
  'trip_id': '00007bf6442eddf62c3e4dbe198fdc4c2e09ccbb',
  'trip_miles': '1',
  'trip_seconds': '480',
  'trip_start_timestamp': '2019-10-28T17:15:00',
  'trip_total': '8.5'}]

#### get_trips(query) method

Since your goal is to look at many different taxi rides, we want to simplify things by creating a method that just takes in the query we want to process. To do this, we simply put the repeated information inside of the method that *wraps* the the actual GET request.

In [29]:
def get_trips(query):
    query["team"] = TEAM
    response = requests.get(URL + "/trips/", params=query)
    return response.json()

In [30]:
# Example usage of get_trips - no need to specify the team or GET request anymore!
get_trips({
        "start": "9/10/2017 3:45 AM",
        "end": "9/10/2017 3:46 AM",
        "limit": 5
    })

{'length': 4,
 'response': [{'company': 'Bauer Taxi Service',
   'dropoff_centroid_latitude': '41.89967018',
   'dropoff_centroid_longitude': '-87.669837798',
   'dropoff_community_area': '24',
   'entry_idx': 0,
   'extras': '1',
   'fare': '8.25',
   'pickup_centroid_latitude': '41.89321636',
   'pickup_centroid_longitude': '-87.63784421',
   'pickup_community_area': '8',
   'taxi_id': '7c51c6dc64becb25ed13b130bf03c67bab1958c6f1a92f6f7fe36f7e0bf499b4b5b5d0c274c663ff6f3d986a48a52a62a12a8935cf5cbbaf2881332500330727',
   'tips': '0',
   'tolls': '0',
   'trip_end_timestamp': '2017-09-10T04:00:00',
   'trip_id': '00151ac4cfa1d6e3debd0d8940ec22a970296f05',
   'trip_miles': '2.2',
   'trip_seconds': '540',
   'trip_start_timestamp': '2017-09-10T03:45:00',
   'trip_total': '9.25'},
  {'company': 'Bauer Taxi Service',
   'dropoff_centroid_latitude': '41.945010151',
   'dropoff_centroid_longitude': '-87.680928225',
   'dropoff_community_area': '5',
   'entry_idx': 1,
   'extras': '0',
   'far

#### Sending SQL queries

The GET request for /trips can accept SQL queries when passes as strings.

<strong>Parameters:</strong>
<ul>
    <li>team</li>
    <li>where : Indicates the SQL WHERE clause which defines the query.</li>
    <li>limit</li>
</ul>

In [31]:
get_trips({
        "where": "(trip_start_timestamp BETWEEN {9/10/2017 2:00 PM} AND {9/10/2017 3:00 PM}) AND (trip_total BETWEEN 10 AND 20)",
        "limit": 1
})

{'length': 0, 'response': [], 'success': True}

NOTE: Times are rounded off to every quarter hour. By specifying that you want the rides in between 3:00PM and 3:15PM, you're also getting back rides from 3:15PM to ~3:23PM. This is because the rides that happened at this time rounded down to 3:15PM.

You might be asking... well, how do we parse this data? We'll take a look at that in the data analysis section.

### /count endpoint

In [32]:
def get_count(query):
    query["team"] = TEAM
    response = requests.get(URL + "/count/", params=query)
    return response.json()

#### /count example : What if our simulation lands on New Year's?

In [33]:
# New Years
new_years_count = get_count({
        "start": "12/31/2016 12:00 AM",
        "end": "01/01/2017 05:00 AM"
    })
print("# of rides in 29-hour span of New Years Eve -> Day: {}".format(new_years_count["count"]))

# of rides in 29-hour span of New Years Eve -> Day: 87776


## Learning Objective: Data Preprocessing and Analysis

Before applying any data analysis to your data, it's always important to preprocess your data. This means looking for any missing values, identifying categorical variables, accounting for outlier (Y) or leverage (X) points. We'll go through the process of getting the data for a specific simulation, processing it with the stack we're familiar with (scipy), and begin some explatory data analysis on it.

In [34]:
# What data can we currently analyze?
requests.get(URL + "/time/").json()

{'message': 'The simulation is not over, the time is 11/3/2019.',
 'sucess': True,
 'time': 1572739200000}

Since the simulation time is 10/1/2017, we can look at data before 10/1/2017 12:00 AM. For this example analysis, we'll look at the week before the simulation time.

In [35]:
# How much data should we expect?
get_count({
        "start": "09/24/2017 12:00 AM",
        "end": "10/1/2017 12:00 AM"
    })

{'count': '596353', 'success': True}

In [36]:
# 1 week before the simulation time ends
get_trips({
        "start": "09/24/2017 12:00 AM",
        "end": "10/1/2017 12:00 AM"
    })["length"]

692

<strong>Well... that's odd.</strong> The call to /count is telling us that we should expect 342,326 taxi rides in the week of 09/24 - 10/01. However, we're only getting results back 717 trips back for the same date range call to /trips. This is because, on the backend, the Lyft RESTful API sets an internal <strong>limit</strong> of 1000 taxi rides.

This presents an interesting problem: how do we get back all of the taxi rides of a specific time range if we're bound to just 1000? Well, we have to use the offset paramater and set it to the length 1000 to get the next set of taxi rides. To get all 342,326 rides, we would have to do this process 342 times. This is simply unreasonable, so instead we need a good <strong>sampling strategy</strong>.

### Converting JSON to Pandas DataFrame

In [37]:
week_before = get_trips({
        "start": "09/24/2017 12:00 AM",
        "end": "10/1/2017 12:00 AM",
        "limit": 100
    })

In [38]:
week_before = week_before["response"]

In [39]:
import pandas as pd
import numpy as np

week_before_dataset = pd.DataFrame(week_before)
week_before_dataset

,company,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_community_area,entry_idx,extras,fare,pickup_centroid_latitude,pickup_centroid_longitude,pickup_community_area,taxi_id,tips,tolls,trip_end_timestamp,trip_id,trip_miles,trip_seconds,trip_start_timestamp,trip_total
0,Bauer Taxi Service,41.944226601,-87.655998182,6,0,2,8.65,41.944226601,-87.655998182,6,af997429ff78eb1fc86738899ca0bd72f50237b56ff82e...,0,0,2017-09-24T04:15:00,000c15faf8f6a75d39d41ec863337a6819c07621,2.6,540,2017-09-24T04:00:00,10.65
1,Bauer Taxi Service,41.92276062,-87.699155343,22,1,0,9.25,41.953582125,-87.72345239,16,51e37146cd6467734e50a00ffe2ebbb77ee35e5f207817...,0,0,2017-09-24T04:15:00,000b75682eb7da7916e97364d94d46bcffd095bb,2.7,600,2017-09-24T04:15:00,9.25
2,Bauer Taxi Service,41.922082541,-87.634156093,7,2,0,8.05,41.892042136,-87.63186395,8,e9b5758569e2721ce79fe9e747a463a6da132f2ebb4854...,1,0,2017-09-24T04:30:00,0008da9f84aed909bccdc920e3cf62b7cdda5c04,2.3,480,2017-09-24T04:15:00,9.05
3,Bauer Taxi Service,41.97907082,-87.903039661,76,3,2,34.85,41.892072635,-87.628874157,8,ffb8cefb331d92d05289bb6d84fba73aac7ed8252c6f4b...,0,0,2017-09-24T06:30:00,000bac8c5564c2bba5f144afd1feb0967fdefa4b,17.5,1080,2017-09-24T06:15:00,36.85
4,Bauer Taxi Service,41.958055933,-87.660389456,3,4,1,7.65,41.958055933,-87.660389456,3,613e73cf1e52bdfb496fb3e51d8c8f140fc0b5610fc962...,3,0,2017-09-24T10:30:00,00082216e5d860d58bf25686df9dd1be7ddef263,1.6,600,2017-09-24T10:15:00,11.65
5,Bauer Taxi Service,41.922082541,-87.634156093,7,5,1,7.25,41.914747305,-87.654007029,7,7c9b9496b28872d456125a7449c7e9dfe32528b375ee5a...,0,0,2017-09-24T11:30:00,0004850f5a61497d922aabbae9ac9a6377c768da,1.5,600,2017-09-24T11:15:00,8.25
6,Bauer Taxi Service,41.891971508,-87.612945414,8,6,1,6.25,41.884987192,-87.620992913,32,2d5029b701200a2ab66bd5071f743efe20a3036597d2b3...,0,0,2017-09-24T16:00:00,0001883840714c8b850d6c681803ae48c1a963c1,1.2,420,2017-09-24T16:00:00,7.25
7,Bauer Taxi Service,41.879066994,-87.657005027,28,7,2,33.65,41.97907082,-87.903039661,76,3f2424dac2387e756c5b4f6e8a5b7705253625cfae6d10...,0,0,2017-09-24T18:15:00,000c9b72a369718d867b6383c79d6f2fea9ecce4,16.4,1560,2017-09-24T17:45:00,35.65
8,Bauer Taxi Service,41.89503345,-87.619710672,8,8,2,36.65,41.97907082,-87.903039661,76,a53d5e8132e54e4852ae9bafbcdbb34eba8db5b5f5c685...,15,0,2017-09-24T19:30:00,0002523f1a462d2a514d78f171f18e69b8db958c,17.8,1680,2017-09-24T19:00:00,53.65
9,Bauer Taxi Service,41.88528132,-87.6572332,28,9,1.5,5.05,41.885300022,-87.642808466,28,b4df572ad07112d99548875cbd778982a6a2918acec411...,3,0,2017-09-24T20:30:00,0001b280e405bf84bc3443abf1cb579965cf8986,0.8,240,2017-09-24T20:30:00,9.55


### Finding the Average "fare"

Let's try to isolate the fare column so we can get the average. It's on index 6, and we can use the <strong>iloc</strong> method to capture it. The <strong>describe()</strong> function will give us summary statistics of the data.

In [53]:
# Let's isolate the "fare" col
week_before_dataset.iloc[:, 6].describe()

count       63
unique      51
top       7.65
freq         3
Name: fare, dtype: object

Well... that's odd. Where's the mean or median output? It turns out that the data returned to us in "fare" is in string format, so we need to convert the column to be a float.

In [54]:
fare_feature = week_before_dataset.iloc[:, [6]].copy()
fare_feature = fare_feature.astype("float64")
fare_feature.head()

,fare
0,8.65
1,9.25
2,8.05
3,34.85
4,7.65


In [55]:
# Confirming that we've converted to a float
fare_feature.dtypes

fare    float64
dtype: object

#### Summary Statistics

In [43]:
fare_feature.describe()

,fare
count,63.000000
mean,12.995873
std,10.127786
min,3.850000
25%,5.850000
50%,8.850000
75%,15.350000
max,39.050000


### Finding the Most Common Zone

Power zones are areas you can configure that require riders to pay an extra cost to take a Lyft, but with the benefit of it coming faster than, say, a taxi. The zone mappings can be seen in the <strong>"dropoff_community_area"</strong> feature. Let's try to find the most common zone, and then simply use that as the only power zone we want to apply for this simulation's pricing model.

In [49]:
week_before_dataset.iloc[:, 3].describe()

count     63
unique    16
top        8
freq      13
Name: dropoff_community_area, dtype: object

In [58]:
week_before_dataset.iloc[:, 9].astype("float64").describe()

count    63.000000
mean     23.730159
std      22.019937
min       3.000000
25%       8.000000
50%      16.000000
75%      32.000000
max      77.000000
Name: pickup_community_area, dtype: float64

Let's take the average fare divided by 2 we found in the last section, coupled with the most frequent zone, and set those as the fares for the Lyft and power zones list, respectively!

In [45]:
def set_pricing(pricing):
    query = pricing;
    query["team"] = TEAM;
    response = requests.post(URL + "/pricing/", params=query)
    return response.json()

In [46]:
def set_zones(zones):
    zone_list = (",").join(str(z) for z in zones)
    query = {
        "team": TEAM,
        "zones": zone_list
    }
    response = requests.post(URL + "/zones/", params=query)
    return response.json()

In [140]:
my_price_model = set_pricing({
    "base": 17.24 / 2,
    "pickup": 0.00,
    "per_mile": 0.00,
    "per_minute": 0.00
})

In [141]:
my_power_zones = set_zones([8])